In [1]:
import pandas as pd
import torch


In [ ]:
%%time

df1 = pd.read_pickle('data/methyl_scores_v1_HM450k_1.pkl', compression="bz2")
df2 = pd.read_pickle('data/methyl_scores_v1_HM450k_2.pkl', compression="bz2")
df3 = pd.read_pickle('data/methyl_scores_v1_HM450k_3.pkl', compression="bz2")
df4 = pd.read_pickle('data/methyl_scores_v1_HM450k_4.pkl', compression="bz2")
df5 = pd.read_pickle('data/methyl_scores_v1_HM450k_5.pkl', compression="bz2")
df = pd.concat([df1, df2, df3, df4, df5], axis=0)


In [ ]:
df.shape

In [ ]:
df[['id', 'geo_accession', 'title', 'sex', 'age', 'race', 'disease',
       'tissue', 'geo_platform', 'inferred_sex', 'inferred_age_Hannum',
       'inferred_age_SkinBlood', 'inferred_age_Horvath353']]

# Prepare the data

In [ ]:
# Assuming `df` is your DataFrame
metadata_columns = ['id', 'geo_accession', 'title', 'sex', 'age', 'race',
                    'tissue', 'geo_platform', 'inferred_age_Hannum',
                    'inferred_age_SkinBlood', 'inferred_age_Horvath353']  # list of metadata columns

label_column = 'disease'  # column with target values for classification/regression
condition_column = 'inferred_sex'
numerical_data = df.drop(metadata_columns + [label_column] + [condition_column], axis=1)  # features for training



Fill in the NA values in the `label_column`

In [ ]:
default_value = 'no_label'
df[label_column].fillna(default_value, inplace=True)

labels = df[label_column]  # target/label for model training
conditions = df[condition_column]  # target/label for model training

In [ ]:
labels.value_counts()

## Detect and drop unreliable columns

In [ ]:
# numerical_data.isna().sum(axis=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# # Calculate the percentage of NaN values in each column
nan_percentage = numerical_data.isna().sum(axis=0) / numerical_data.shape[0] * 100

# Plot the histogram of the percentage of NaN values per column
plt.figure(figsize=(10, 6))
plt.hist(nan_percentage, bins=50, edgecolor='k', alpha=0.7)
plt.title("Histogram of Percentage of NaN Values Per Column")
plt.xlabel("Percentage of NaN values")  
plt.ylabel("Number of Columns")
plt.grid(True)

plt.show()


In [ ]:
print(f"more than 1% NaN: {(nan_percentage>1).sum()}")
print(f"more than 5% NaN: {(nan_percentage>5).sum()}")
print(f"more than 10% NaN: {(nan_percentage>10).sum()}")
print(f"more than 15% NaN: {(nan_percentage>15).sum()}")
print(f"more than 20% NaN: {(nan_percentage>20).sum()}")
print(f"more than 30% NaN: {(nan_percentage>30).sum()}")


In [ ]:
# Select subset of columns where NaN percentage is less than 10%
selected_columns = nan_percentage[nan_percentage < 10].index.tolist()

# Create a new DataFrame with the selected columns
numerical_data_filtered = numerical_data[selected_columns]



In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Convert categorical labels to one-hot vectors
labels_onehot = onehot_encoder.fit_transform(labels.values.reshape(-1, 1))



In [ ]:
labels_onehot.shape

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler

SCALE = True
## With SCALE = False, the loss would explode! therefore, we scale the data to control the range of the loss and the gradients
if SCALE:
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(numerical_data_filtered)
else:
    features_scaled = numerical_data_filtered.values

In [ ]:
%%time
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

X = features_scaled
y = labels_onehot

# Stratified shuffle split
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in splitter.split(X, y):
    X_train, X_temp = X[train_index], X[test_index]
    y_train, y_temp = y[train_index], y[test_index]

# Split the temp set into validation and test sets (15% val, 15% test)
splitter_val_test = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for val_index, test_index in splitter_val_test.split(X_temp, y_temp):
    X_val, X_test = X_temp[val_index], X_temp[test_index]
    y_val, y_test = y_temp[val_index], y_temp[test_index]



In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the stratified split
print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size: {X_test.shape}")


# Build VAE

In [ ]:
import os

# Set the environment variable inside the script
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=256):
        super(VAE, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2_mu = nn.Linear(hidden_dim, latent_dim)  # for mean
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)  # for log variance
        
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
    
    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc2_mu(h)
        logvar = self.fc2_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        # Check if logvar has NaN or Inf values
        if torch.isnan(logvar).any() or torch.isinf(logvar).any():
            print(f"NaN or Inf detected in logvar: logvar={logvar}")
        
        # Clamp logvar to prevent extreme values
        logvar = torch.clamp(logvar, min=-10, max=10)
        
        # Calculate std from logvar
        std = torch.exp(0.5 * logvar)
        
        # Check if std has NaN or Inf values
        if torch.isnan(std).any() or torch.isinf(std).any():
            print(f"NaN or Inf detected in std computation: std={std}")
        
        # Sample from the latent space
        eps = torch.randn_like(std)
        z = mu + eps * std
        
        # Check if z has NaN or Inf values
        if torch.isnan(z).any() or torch.isinf(z).any():
            print(f"NaN or Inf detected in z computation: z={z}")
        
        return z

    def decode(self, z):
        h = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

    def get_latent_embedding(self, x):
        """
        Method to get the latent embedding (the `z` vector) for an input.
        """
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)  # this is the embedding
        return z


In [ ]:
class VAE_Lightning(pl.LightningModule):
    def __init__(self, input_dim=485577, latent_dim=100, hidden_dim=512, lr=1e-3):
        super(VAE_Lightning, self).__init__()
        self.model = VAE(input_dim, latent_dim, hidden_dim)
        self.lr = lr
    
    def forward(self, x):
        mu, logvar = self.model.encode(x)
        z = self.model.reparameterize(mu, logvar)
        return z, mu, logvar

    def get_latent_embedding(self, x):
        return self.model.get_latent_embedding(x)
        
    def training_step(self, batch, batch_idx):
        x, _ = batch

        # Step 1: Create mask before replacing NaN values
        mask = ~torch.isnan(x)  # mask where values are not NaN

        # Step 2: Replace NaNs with zero or another neutral value for forward pass
        x_filled = torch.nan_to_num(x, nan=0.0)

        # Step 3: Pass through the model with filled values
        z, mu, logvar = self.forward(x_filled)
        x_hat, _, _ = self.model(x_filled)

        # Step 4: Use the original x (with NaNs) and mask to calculate the loss
        loss = self._vae_loss(x, x_hat, mu, logvar, mask)
        print(f"Training loss: {loss.item()}")

        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    
    def validation_step(self, batch, batch_idx):
        x, _ = batch

        # Step 1: Create mask before replacing NaN values
        mask = ~torch.isnan(x)

        # Step 2: Replace NaNs with zero or another neutral value for forward pass
        x_filled = torch.nan_to_num(x, nan=0.0)

        # Step 3: Pass through the model with filled values
        z, mu, logvar = self.forward(x_filled)
        x_hat, _, _ = self.model(x_filled)

        # Step 4: Use the original x (with NaNs) and mask to calculate the loss
        loss = self._vae_loss(x, x_hat, mu, logvar, mask)
        print(f"Validation loss: {loss.item()}")

        self.log('val_loss', loss, on_step=False, on_epoch=True)
  

    def _vae_loss(self, original_x, x_hat, mu, logvar, mask):
        # Apply mask to ignore NaN values in the loss calculation
        recon_loss = F.mse_loss(x_hat[mask], original_x[mask], reduction='mean')
    
        # Scale the KL divergence to balance the losses
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        kl_loss = kl_loss / original_x.shape[0]  # Normalize by batch size or apply weighting
    
        return recon_loss + kl_loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

# Train the model

In [ ]:
# %%time

# # Extract embeddings for the entire dataset
# def extract_embeddings(model, dataloader):
#     model.eval()
#     embeddings = []
#     with torch.no_grad():
#         for batch in dataloader:
#             x, _ = batch
#             z, _, _ = model.forward(x)  # Get the latent embedding z
#             embeddings.append(z)
#     return torch.cat(embeddings, dim=0)

# # Train the VAE model first
# trainer = Trainer(max_epochs=10, gpus=1 if torch.cuda.is_available() else 0)
# trainer.fit(model, train_loader, val_loader)

# # Extract latent embeddings for training and validation sets
# train_embeddings = extract_embeddings(model, train_loader)
# val_embeddings = extract_embeddings(model, val_loader)

# # Convert embeddings to numpy for later use
# train_embeddings_np = train_embeddings.cpu().numpy()
# val_embeddings_np = val_embeddings.cpu().numpy()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
%%time
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

# Initialize the VAE Lightning model
input_dim = X_train_tensor.shape[1]  # The number of input features
latent_dim = 100  # Latent dimension size, can be tuned
model = VAE_Lightning(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=1000, lr=1e-6)

# Training

checkpoint_callback = ModelCheckpoint(monitor='val_loss', save_top_k=1, mode='min')

trainer = pl.Trainer(
    max_epochs=10,
    gradient_clip_val=0.5,  # Clip gradients to avoid explosion
    callbacks=[checkpoint_callback],
    precision=32,
    accelerator='gpu',          # Use 'gpu' or 'cpu'
    devices=1 if torch.cuda.is_available() else 'auto',  # Use 1 GPU or CPU ('auto' will pick the appropriate one)
)
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | VAE  | 904 M  | train
---------------------------------------
904 M     Trainable params
0         Non-trainable params
904 M     Total params
3,616.992 Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 512/512 [01:38<00:00,  5.20it/s, v_num=11]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/110 [00:00<?, ?it/s]Validation loss: 3.280782461166382

Validation DataLoader 0:   1%|          | 1/110 [00:00<00:02, 49.16it/s]Validation loss: 1.6873259544372559

Validation DataLoader 0:   2%|▏         | 2/110 [00:00<00:02, 39.46it/s]Validation loss: 1.9775679111480713

Validation DataLoader 0:   3%|▎         | 3/110 [00:00<00:03, 35.21it/s]Validation loss: 1.6448919773101807

Validation DataLoader 0:   4%|▎         | 4/110 [00:00<00:03, 33.31it/s]Validation loss: 1.5286903381347656

Validation DataLoader 0:   5%|▍         | 5/110 [00:00<00:03, 32.05it/s]Validation loss: 2.2390265464782715

Validation DataLoader 0:   5%|▌         | 6/110 [00:00<00:03, 31.44it/s]Validation loss: 2.3711185455322266

Validation DataLoader 0:   6%|▋         | 7/110 [00:00<00:03, 31.60it/s]Validation loss: 1.7049884796142578

Validation DataLoader 

In [ ]:
from pytorch_lightning import Trainer

checkpoint_path = "lightning_logs/version_11/checkpoints/epoch=9-step=5120.ckpt"

loaded_model = VAE_Lightning.load_from_checkpoint(
    checkpoint_path,
    input_dim=input_dim,
    latent_dim=latent_dim,
    hidden_dim=1000,
    map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

loaded_model.eval()

In [ ]:
def get_latent_embeddings(model, dataloader):
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            x,y = batch

            x = x.to(device)
            y = y.to(device)
            
            # Replace NaNs with zero or another neutral value for forward pass
            x_filled = torch.nan_to_num(x, nan=0.0)
            
            z, _, _ = model.forward(x_filled)
            embeddings.append(z)
            labels.append(y)
        
        embeddings = torch.cat(embeddings, dim=0)
        labels = torch.cat(labels, dim=0)

    return embeddings, labels

train_embeddings, train_labels = get_latent_embeddings(loaded_model, train_loader)
val_embeddings, val_labels = get_latent_embeddings(loaded_model, val_loader)
test_embeddings, test_labels = get_latent_embeddings(loaded_model, test_loader)

        

In [44]:
train_embeddings.shape

torch.Size([8185, 100])

In [52]:
import umap
import matplotlib.pyplot as plt
import numpy as np

# Combine train and validation embeddings
combined_embeddings = torch.cat([train_embeddings, val_embeddings], dim=0).cpu().numpy()
combined_labels = torch.cat([train_labels, val_labels], dim=0).cpu().numpy()

# Fit UMAP on the combined embeddings
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean', random_state=42)
combined_umap = umap_model.fit_transform(combined_embeddings)

# Plot UMAP embeddings (train + validation)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(combined_umap[:, 0], combined_umap[:, 1], c=combined_labels, cmap='Spectral', s=10, alpha=0.8)
plt.colorbar(scatter)
plt.title('UMAP Projection of Train + Validation Embeddings')
plt.show()


ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.

In [53]:
import umap

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.